<a href="https://colab.research.google.com/github/spatiebalk/text_mining_project/blob/master/text_generation_GRU_hp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text generation GRU with Harry Potter text


In [1]:
import numpy as np
import tensorflow as tf
import os
from os.path import join, isfile
import time
import csv
from tqdm import tqdm
import json
from os import listdir
from os.path import isfile, join
import sys

In [2]:
import sys
print(sys.version)
print(tf.__version__)

3.6.9 (default, Oct  8 2020, 12:12:24) 
[GCC 8.4.0]
2.4.0


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


### Harry Potter data

In [15]:
text = open('/content/gdrive/My Drive/TxMM/harrypotter.txt').read()
print(len(text))

6251649


### 1. GRU
From https://medium.com/towards-artificial-intelligence/create-your-own-harry-potter-short-story-using-rnn-and-tensorflow-853b3ed1b8f3 and https://www.tensorflow.org/tutorials/text/text_generation

In [5]:
# print(text[:200])

In [6]:
vocab = sorted(set(text))
char2index = {u:i for i, u in enumerate(vocab)}
index2char = np.array(vocab)
text_as_int = np.array([char2index[c] for c in text])

In [7]:
seq_length = 50
examples_per_epoch = len(text)//(seq_length+1)
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

def split_input_target(data):
  input_text = data[:-1]
  target_text = data[1:]
  return input_text, target_text

dataset = sequences.map(split_input_target)

BATCH_SIZE = 64
BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [8]:
vocab_size = len(vocab)
embedding_dim = 300

# Number of RNN units 
rnn_units1 = 512
rnn_units2 = 256
rnn_units= [rnn_units1, rnn_units2]

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
       batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units1, return_sequences=True,
       stateful=True,recurrent_initializer='glorot_uniform'),
    tf.keras.layers.GRU(rnn_units2, return_sequences=True,
       stateful=True,recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(
vocab_size = vocab_size,
embedding_dim=embedding_dim,
rnn_units=rnn_units,
batch_size=BATCH_SIZE)

def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels,
         logits, from_logits=True)
  
model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 300)           31800     
_________________________________________________________________
gru (GRU)                    (64, None, 512)           1250304   
_________________________________________________________________
gru_1 (GRU)                  (64, None, 256)           591360    
_________________________________________________________________
dense (Dense)                (64, None, 106)           27242     
Total params: 1,900,706
Trainable params: 1,900,706
Non-trainable params: 0
_________________________________________________________________


In [9]:
# Directory where the checkpoints will be saved
checkpoint_dir = '/content/gdrive/My Drive/TxMM/GRU_results/training_checkpoints_GRU_hp'

# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")
checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
   filepath=checkpoint_prefix, save_weights_only=True)

In [10]:
EPOCHS= 50
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])
latest_check = tf.train.latest_checkpoint(checkpoint_dir)

Epoch 1/15
1915/1915 [==============================] - 51s 22ms/step - loss: 2.0186 - accuracy: 0.4406
Epoch 2/15
1915/1915 [==============================] - 44s 23ms/step - loss: 1.3350 - accuracy: 0.6029
Epoch 3/15
1915/1915 [==============================] - 44s 23ms/step - loss: 1.2780 - accuracy: 0.6177
Epoch 4/15
1915/1915 [==============================] - 44s 23ms/step - loss: 1.2514 - accuracy: 0.6245
Epoch 5/15
1915/1915 [==============================] - 44s 22ms/step - loss: 1.2344 - accuracy: 0.6290
Epoch 6/15
1915/1915 [==============================] - 44s 22ms/step - loss: 1.2243 - accuracy: 0.6315
Epoch 7/15
1915/1915 [==============================] - 44s 22ms/step - loss: 1.2165 - accuracy: 0.6337
Epoch 8/15
1915/1915 [==============================] - 44s 22ms/step - loss: 1.2110 - accuracy: 0.6350
Epoch 9/15
1915/1915 [==============================] - 44s 23ms/step - loss: 1.2072 - accuracy: 0.6362
Epoch 10/15
1915/1915 [==============================] - 44s 23m

In [11]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(latest_check)
model.build(tf.TensorShape([1, None]))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 300)            31800     
_________________________________________________________________
gru_2 (GRU)                  (1, None, 512)            1250304   
_________________________________________________________________
gru_3 (GRU)                  (1, None, 256)            591360    
_________________________________________________________________
dense_1 (Dense)              (1, None, 106)            27242     
Total params: 1,900,706
Trainable params: 1,900,706
Non-trainable params: 0
_________________________________________________________________


In [12]:
def generate_text(model, start_string):

  num_generate = 1000  #can be anything you like
  input_eval = [char2index[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)
  text_generated = []
  scaling = 0.5 #kept at a lower value here
  # Here batch size == 1
  model.reset_states()

  for i in range(num_generate):
    predictions = model(input_eval)
    # remove the batch dimension
    predictions = tf.squeeze(predictions, 0)

    # using a categorical distribution to predict the character returned by the model
    predictions = predictions / scaling
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    # Pass the predicted character as the next input to the model
    # along with the previous hidden state
    input_eval = tf.expand_dims([predicted_id], 0)

    text_generated.append(index2char[predicted_id])

  return (start_string + "".join(text_generated))

In [13]:
print(generate_text(model, start_string= "Severus Snape"))

Severus Snape looked around at him for a moment or two hand and continued to see whether a boy who had not seen the truth in the middle of the top of the thick in the smashed and times in his pocket, and his pale battle was blasted on the sky and returned to the brilliance with him that the golden landing was obscured by the distant particular staircase. The same to help the right now that he was sure he was still standing there. He was so tired with the source of the other side of the same time, as the place where the dead look at him. He was back at the same time. And the great scar on the back to the corridor outside the staircase. He was still standing at the same tears. Harry saw that he had seen her that the teachers were not sure that the continued carved the place where the fire was still there were speak to his grave, and the way the boy had a bit of that. The wand pointed at Lupin began, and he shook him at the way that he had vanished. Harry saw what you are forgotten that t

In [14]:
# open prompts file
prompts = open('/content/gdrive/My Drive/TxMM/hp_prompts.txt', 'r').readlines()
generated_dict = {}

for i, p in tqdm(enumerate(prompts)):
  # generate 1000 characters
  gen_text = generate_text(model, start_string=p)
  generated_dict[i] = gen_text

with open('/content/gdrive/My Drive/TxMM/GRU_results/hp_generated_texts.json', 'w') as fp:
  json.dump(generated_dict, fp)

96it [10:07,  6.24s/it]

KeyboardInterrupt: ignored